# AFQMC

蚂蚁金融语义相似度 Ant Financial Question Matching Corpus

## 训练

In [ ]:
from pyclue.tf1.tasks.sentence_pair.siamese.train import Trainer

In [ ]:
# 数据所在路径
data_dir = '/workspace/projects/PyCLUE_Corpus/sentence_pair/afqmc'
# 训练结果保存路径
output_dir = '/workspace/projects/PyCLUE_examples/afqmc'

# 是否重新创建tfrecord（会耗费一定时间，首次执行时需设置为True；当设为False时，可保证数据一致性）
recreate_tfrecord = True
# 随机种子
random_seed = 0

# 使用内置的预训练语言模型时，需指定model_name，无需指定model_type, vocab_file, config_file和init_checkpoint_file
# 使用自定义预训练语言模型时，需指定model_type, vocab_file, config_file和init_checkpoint_file，无需指定model_name
model_name = 'albert_tiny_zh_brightmart'
model_type = None
vocab_file = None
config_file = None
init_checkpoint_file = None

# 训练时能接受的最长句长，注：该句长不能超过预训练时所指定的最大句长
max_seq_len = 128
# 训练步数(num_train_steps) = 训练数据量(num_train_examples) * 训练轮次(num_train_epochs) / 每批次训练数据大小(batch_size)
# 预热步数(num_warmup_steps) = 训练步数(num_train_steps) * 预热比例(warmup_proportion)
# 训练轮次
num_train_epochs = 3
# 预热比例
warmup_proportion = 0.1
# 每批次训练数据大小
batch_size = 16

# 初始学习率
learning_rate = 2e-5
# 训练器名，可选adamw和lamb
optimizer_name = 'adamw'

# 验证指标，可选accuracy, premise, recall, f1
metric_name = 'accuracy'

# 每若干训练步数保存一次checkpoint模型（最多保存最新的10个模型）
save_checkpoints_steps = 200
# 每若干训练步数打印一次训练指标
log_steps = 50
# 训练与验证执行策略，可选0（异步：先训练后验证，较快，CLUE默认采用此方式）或1（同步：同时执行训练与验证，较慢）
train_and_evaluate_mode = 0
# 是否使用最佳的checkpoint进行验证，当为True时，从已保存的checkpoint模型中选择最佳的模型进行预测；
# 当为False时，使用最后一个checkpoint模型进行预测；仅在train_and_evaluate_mode=0时有效，CLUE默认采用False
apply_best_checkpoint = False
# early stop参数，执行若干步时指标任未提升时停止训练模型，仅在train_and_evaluate_mode=1时有效
max_steps_without_increase = 500
# early stop参数，最低训练若干步时才执行early stop策略，仅在train_and_evaluate_mode=1时有效
min_steps = 500

In [ ]:
# 初始化训练器
trainer = Trainer(
    output_dir=output_dir, 
    random_seed=random_seed)
# 构建模型
trainer.build_model(
    model_name=model_name,
    model_type=model_type,
    vocab_file=vocab_file,
    config_file=config_file,
    init_checkpoint_file=init_checkpoint_file,
    max_seq_len=max_seq_len)
# 加载数据
trainer.load_data(
    data_dir=data_dir,
    batch_size=batch_size,
    recreate_tfrecord=recreate_tfrecord)
# 执行训练并输出保存好的模型路径（包含checkpoint模型和pb模型）
model_file_dict = trainer.train_and_evaluate(
    num_train_epochs=num_train_epochs,
    warmup_proportion=warmup_proportion,
    learning_rate=learning_rate,
    optimizer_name=optimizer_name,
    log_steps=log_steps,
    metric_name=metric_name,
    save_checkpoints_steps=save_checkpoints_steps,
    max_steps_without_increase=max_steps_without_increase,
    min_steps=min_steps,
    mode=train_and_evaluate_mode,
    apply_best_checkpoint=apply_best_checkpoint)

In [ ]:
print('model save path: \n%s' % '\n'.join(['%s: %s' % item for item in model_file_dict.items()]))